<a href="https://colab.research.google.com/github/rahiakela/fluent-python-book-practice/blob/master/part-iv-object-oriented-idioms/10_sequence_hacking_hashing_and_slicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector: a user-defined sequence type

In this notebook we will create a class to represent a multi-dimensional Vector class that will behave like a standard Python immutable flat sequence. Its elements will be floats, and it will support the following by the end of this chapter:

- Basic sequence protocol: `__len__` and `__getitem__`.
- Safe representation of instances with many items.
- Proper slicing support, producing new Vector instances.
- Aggregate hashing taking into account every contained element value.
- Custom formatting language extension.

We’ll also implement dynamic attribute access with __getattr__ as a way of replacing the read-only properties we used in Vector2d — although this is not typical of sequence types.

We’ll talk about how protocols and duck typing are related, and its practical implications when you create your own types.

Our strategy to implement Vector will be to use composition, not inheritance. We’ll store the components in an array of floats, and will implement the methods needed for our Vector to behave like an immutable flat sequence

## Who needs a vector with 1000-dimensions?

N-dimensional vectors (with large values of N) are widely used in
information retrieval, where documents and text queries are represented as vectors, with one dimension per word. This is called the **Vector space model**. 

In this model, a key relevance metric is the cosine similarity, i.e. the cosine of the angle between a query vector and a document vector. As the angle decreases the cosine approaches the maximum value of 1, and so does the relevance of the document to the query.

**NumPy** and **SciPy** are the tools you need for real world vector math. The PyPI package gemsim, by Radim Rehurek, implements vector space modeling for **natural language processing** and information retrieval, using NumPy and SciPy.

## Vector take #1: Vector2d compatible

We could make `Vector(3, 4)` and `Vector(3, 4, 5)` work, by taking arbitrary arguments with *args in `__init__`, but the best practice for a sequence constructor is to take the data as an iterable argument in the constructor, like all built-in sequence types do.

In [2]:
from array import array
import math
import reprlib

In [9]:
class Vector:
  # typecode is a class attribute we’ll use when converting Vector2d instances to/from bytes.
  typecode = 'd'

  def __init__(self, components):
    # protected attribute will hold an array with the Vector components.
    self._components = array(self.typecode, components)

  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation
    components = reprlib.repr(self._components)
    # Remove the array('d', prefix and the trailing ) before plugging the string into a Vector constructor call.
    components = components[components.find("["):-1]
    return 'Vector({})'.format(components)

  def __str__(self):
    # From an iterable Vector2d it’s easy to build a tuple for display as an ordered pair.
    return str(tuple(self))

  def __bytes__(self):
    # Build a bytes object directly from self._components.
    return (bytes([ord(self.typecode)]) + bytes(self._components))

  def __eq__(self, other):
    # To quickly compare all components, build tuples out of the operands.
    return tuple(self) == tuple(other)

  def __abs__(self):
    # We can’t use hypot anymore, so we sum the squares of the components and compute the sqrt of that.
    return math.sqrt(sum(x * x for x in self))

  def __bool__(self):
    # uses abs(self) to compute the magnitude, then converts it to bool
    return bool(abs(self))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    # we pass the memoryview directly to the constructor, without unpacking with * as we did before.
    return cls(memv)

In [10]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [11]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [12]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

By the way, we could have sub-classed Vector from Vector2d but I chose not to do it for two reasons. First, the incompatible constructors really make sub-classing not advisable. I could work around that with some clever parameter handling in `__init__`, but the second reason is more important: I want Vector to be a stand-alone example of a class implementing the sequence protocol.

### Protocols and duck typing

We don’t need to inherit from any special class to create a fully functional sequence type in Python, you just need to implement the methods
that fulfill the sequence protocol. But what kind of protocol are we talking about?

In the context of Object Oriented Programming, a protocol is an informal interface, defined only in documentation and not in code. For example, the sequence protocol in Python entails just the `__len__` and `__getitem__` methods. 

Any class Spam that implements those methods with the standard signature and semantics can be used anywhere a sequence is expected. Whether Spam is a subclass of this or that is irrelevant, all that matters is that it provides the necessary methods.

In [13]:
import collections

In [15]:
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

This class takes advantage of many Python facilities because it implements the sequence protocol, even if that is not declared anywhere in the code.

Any experienced Python coder will look at it and understand that it is a sequence, even if it subclasses object. We say it is a sequence because it behaves like one, and that is what matters.

Because protocols are informal and unenforced, you can often get away implementing just part of a protocol, if you know the specific context where a class will be used. 

For example, to support iteration, only `__getitem__` is required, there is no need to provide `__len__`.

## Vector take #2: a sliceable sequence

Supporting the sequence protocol is really easy if you can delegate to a sequence attribute in your object, like our `self._components` array. These `__len__` and `__getitem__` one-liners are a good start:

In [16]:
class Vector:
  # typecode is a class attribute we’ll use when converting Vector2d instances to/from bytes.
  typecode = 'd'

  def __init__(self, components):
    # protected attribute will hold an array with the Vector components.
    self._components = array(self.typecode, components)

  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation
    components = reprlib.repr(self._components)
    # Remove the array('d', prefix and the trailing ) before plugging the string into a Vector constructor call.
    components = components[components.find("["):-1]
    return 'Vector({})'.format(components)

  def __str__(self):
    # From an iterable Vector2d it’s easy to build a tuple for display as an ordered pair.
    return str(tuple(self))

  def __bytes__(self):
    # Build a bytes object directly from self._components.
    return (bytes([ord(self.typecode)]) + bytes(self._components))

  def __eq__(self, other):
    # To quickly compare all components, build tuples out of the operands.
    return tuple(self) == tuple(other)

  def __abs__(self):
    # We can’t use hypot anymore, so we sum the squares of the components and compute the sqrt of that.
    return math.sqrt(sum(x * x for x in self))

  def __bool__(self):
    # uses abs(self) to compute the magnitude, then converts it to bool
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, index):
    return self._components[index]

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    # we pass the memoryview directly to the constructor, without unpacking with * as we did before.
    return cls(memv)

In [18]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [19]:
v1[0], v1[-1]

(3.0, 5.0)

In [20]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

As you can see, even slicing is supported — but not very well. It would be better if a slice of a Vector was also a Vector instance and not a array.

Consider the built-in sequence types: every one of them, when sliced, produces a new instance of its own type, and not of some other type.

To make Vector produce slices as Vector instance, we can’t just delegate the slicing to array. We need to analyze the arguments we get in `__getitem__` and do the right thing.

### How slicing works

Checking out the behavior of `__getitem__` and slices.

In [21]:
class MySeq:
  def __getitem__(self, index):
    return index

In [22]:
s = MySeq()
s[1]

1

In [23]:
s[1:4]

slice(1, 4, None)

In [24]:
# slice(1, 4, 2) means start at 1, stop at 4, step by 2.
s[1:4:2]

slice(1, 4, 2)

In [25]:
# __getitem__ receives a tuple.
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [26]:
# The tuple may even hold several slice objects.
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

In [27]:
# Checking out the behavior of __getitem__ and slices.
slice

slice

In [28]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

### A slice-aware `__getitem__`

In our Vector code we’ll not need the slice.indices() method because when we get a slice argument we’ll delegate its handling to the _components array. But if you can’t count on the services of an underlying sequence, this method can be a huge time saver.

Now that we know how to handle slices, let’s see how the improved Vector.`__getitem__` implementation looks like.

The following two methods needed to make Vector behave as a sequence:
`__len__` and `__getitem__` — the latter now implemented to handle slicing correctly.

In [31]:
import numbers

class Vector:
  # typecode is a class attribute we’ll use when converting Vector2d instances to/from bytes.
  typecode = 'd'

  def __init__(self, components):
    # protected attribute will hold an array with the Vector components.
    self._components = array(self.typecode, components)

  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation
    components = reprlib.repr(self._components)
    # Remove the array('d', prefix and the trailing ) before plugging the string into a Vector constructor call.
    components = components[components.find("["):-1]
    return 'Vector({})'.format(components)

  def __str__(self):
    # From an iterable Vector2d it’s easy to build a tuple for display as an ordered pair.
    return str(tuple(self))

  def __bytes__(self):
    # Build a bytes object directly from self._components.
    return (bytes([ord(self.typecode)]) + bytes(self._components))

  def __eq__(self, other):
    # To quickly compare all components, build tuples out of the operands.
    return tuple(self) == tuple(other)

  def __abs__(self):
    # We can’t use hypot anymore, so we sum the squares of the components and compute the sqrt of that.
    return math.sqrt(sum(x * x for x in self))

  def __bool__(self):
    # uses abs(self) to compute the magnitude, then converts it to bool
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, index):
    # Get the class of the instance (i.e. Vector) for later use.
    cls = type(self)
    if isinstance(index, slice):   # If the index argument is a slice…
      # …invoke the class to build another Vector instance from a slice of the _components array.
      return cls(self._components[index])
    elif isinstance(index, numbers.Integral):  # If the index is an int or some other kind of integer…
      # …just return the specific item from _components.
      return self._components[index]
    else:  # Otherwise, raise an exception.
      msg = "{cls.__name__} indices must be integers"
      raise TypeError(msg.format(cls=cls))

    return self._components[index]

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    # we pass the memoryview directly to the constructor, without unpacking with * as we did before.
    return cls(memv)

To discover which exception to raise in the else clause of `__getitem__`, I used the interactive console to check the result of 'ABC'[1, 2]. I then learned that Python raises a TypeError, and I also copied the wording from the error message: “indices must be integers”. To create Pythonic objects, mimic Python’s own objects.

In [32]:
v7 = Vector(range(7))
# An integer index retrieves just one component value as a float.
v7[-1]

6.0

In [33]:
# A slice index creates a new Vector.
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [34]:
# A slice of len == 1 also creates a Vector.
v7[-1:]

Vector([6.0])

In [35]:
# Vector does not support multi-dimensional indexing, so a tuple of indices or slices raises an error.
v7[1, 2]

TypeError: ignored

## Vector take #3: dynamic attribute access Vector